In [1]:
%reload_ext autoreload
%autoreload 2

# Load the data

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

In [3]:
from Scratch import metadata_loader
import numpy as np
from tqdm import tqdm

DEVICE = "ibmq_mumbai"
LOGICAL = str(0)

# Load the metadata
md = metadata_loader(True, True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md.dropna(subset=["rounds"])
md = md[md["meas_level"] == 1]
md['rounds'] = md['rounds'].astype(int)
md['distance'] = md['distance'].astype(int)

md = md[md["backend_name"] == DEVICE]
md = md[md["logical"] == 0]
md = md[md["rounds"] == 3]
md = md[md["distance"] == 3]
md

,creation_date,notebook_name,backend_name,job_id,tags,meas_level,shots,tags_xp,rep_delay,sampled_state,...,optimization_level,code,distance,rounds,logical,layout,descr,num_CNOTS_layers,num_CNOT_layers,resets
2607,2024-03-10 18:36:49.567000+01:00,mean_pos_mumbai,ibmq_mumbai,cqpyzcd2b6ng008y69pg,[RepCode Mean Positions],1.0,6944.0,NaN,0.0005,NaN,...,NaN,RepetitionCodeCircuit,3,3,0,NaN,Mean positions,NaN,NaN,False


# Analyze the job

In [28]:
job = provider.retrieve_job(md.iloc[0]["job_id"])

In [23]:
print(len(job.initial_layouts()))
layout = job.initial_layouts()[8]
print(layout)
print(job.deserialize_layout(layout))


16
{"Qubit(QuantumRegister(22, 'ancilla'), 0)": '0', "Qubit(QuantumRegister(2, 'link_qubit'), 1)": '1', "Qubit(QuantumRegister(3, 'code_qubit'), 1)": '2', "Qubit(QuantumRegister(2, 'link_qubit'), 0)": '3', "Qubit(QuantumRegister(3, 'code_qubit'), 2)": '4', "Qubit(QuantumRegister(3, 'code_qubit'), 0)": '5', "Qubit(QuantumRegister(22, 'ancilla'), 1)": '6', "Qubit(QuantumRegister(22, 'ancilla'), 2)": '7', "Qubit(QuantumRegister(22, 'ancilla'), 3)": '8', "Qubit(QuantumRegister(22, 'ancilla'), 4)": '9', "Qubit(QuantumRegister(22, 'ancilla'), 5)": '10', "Qubit(QuantumRegister(22, 'ancilla'), 6)": '11', "Qubit(QuantumRegister(22, 'ancilla'), 7)": '12', "Qubit(QuantumRegister(22, 'ancilla'), 8)": '13', "Qubit(QuantumRegister(22, 'ancilla'), 9)": '14', "Qubit(QuantumRegister(22, 'ancilla'), 10)": '15', "Qubit(QuantumRegister(22, 'ancilla'), 11)": '16', "Qubit(QuantumRegister(22, 'ancilla'), 12)": '17', "Qubit(QuantumRegister(22, 'ancilla'), 13)": '18', "Qubit(QuantumRegister(22, 'ancilla'), 14)

In [22]:
print(job.result().get_memory(11).shape)

(6944, 9)


# Decode

In [25]:
import json

import pymatching
import stim 
from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, find_longest_path_in_hex, get_repcode_IQ_map
import cpp_soft_info
from Scratch import load_calibration_memory, create_or_load_kde_grid

_DETAILED = False

rel_error = 1
_RESETS = False

# KDE BANDWIDTHS
lin = [0.4, 0.7, 20]
num_points = 51

bandwidths = np.linspace(lin[0], lin[1], lin[2])


# Get the date and noise
date = md.iloc[0]["creation_date"]
noise_dict = get_noise_dict_from_backend(provider, DEVICE, date=date)

# Get the KDEs
all_memories = load_calibration_memory(provider, md.iloc[0]["job_id"])
kde_dict = cpp_soft_info.get_KDEs(all_memories, bandwidths, relError=1, absError=-1, num_points=num_points)

19:15:25 Warning: Could not get two gate error of ECR due to 'Could not find the desired property for ecr', taking CX instead.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 49
Specified job execution date: 2024-03-10 18:37:54.874642+01:00
Found jobs for backend ibmq_mumbai with closest execution date 2024-03-10 16:56:34.510733+00:00.


In [36]:
layouts = list(job.initial_layouts())

num_errors = []
for idx, layout_dict in tqdm(enumerate(layouts), desc="Decoding circuits"):

    layout_dict_des = job.deserialize_layout(layout_dict)

    layout = list(layout_dict_des['link_qubit'].values()) + list(layout_dict_des['code_qubit'].values())

    qubit_mapping = get_repcode_IQ_map(layout, synd_rounds=3)
    avgs = get_avgs_from_dict(noise_dict, layout)

    # Initialize the stim model
    circuit = stim.Circuit.generated("repetition_code:memory",
                            distance=3,
                            rounds=3,
                            after_clifford_depolarization=avgs["two_gate"], #two-qubit-fidelity,
                            after_reset_flip_probability=0 if not _RESETS else avgs["readout"], #reset error
                            before_measure_flip_probability=avgs["readout"], #measurement error,
                            before_round_data_depolarization=avgs["idle"]) #idle error)
    model = circuit.detector_error_model(decompose_errors=False)

    memory = job.result().get_memory(idx)
    result_kde = cpp_soft_info.decode_IQ_kde(model, memory, 3, int(LOGICAL), _RESETS, 
                                                qubit_mapping, kde_dict, _DETAILED, relError=rel_error, absError=-1,
                                                nb_intervals=-1)
    
    num_errors.append(result_kde.num_errors)

print(f"num_errors: {num_errors}")

Decoding circuits: 0it [00:00, ?it/s]

[25, 19, 26, 22, 16]


Decoding circuits: 1it [00:01,  1.60s/it]

[22, 16, 25, 19, 14]


Decoding circuits: 2it [00:02,  1.46s/it]

...

[10, 15, 7, 12, 18]


Decoding circuits: 15it [00:21,  1.48s/it]

[12, 18, 10, 15, 21]
[15, 21, 12, 18, 23]


Decoding circuits: 16it [00:23,  1.46s/it]

num_errors: [10, 16, 19, 30, 33, 22, 54, 9, 40, 5, 34, 16, 47, 30, 12, 59]


In [35]:
import cProfile

def f():
    num_errors = []
    for idx, layout_dict in tqdm(enumerate(job.initial_layouts()), desc="Decoding circuits"):

        layout_dict_des = job.deserialize_layout(layout_dict)

        layout = list(layout_dict_des['link_qubit'].values()) + list(layout_dict_des['code_qubit'].values())

        qubit_mapping = get_repcode_IQ_map(layout, synd_rounds=3)
        avgs = get_avgs_from_dict(noise_dict, layout)

        # Initialize the stim model
        circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=3,
                                rounds=3,
                                after_clifford_depolarization=avgs["two_gate"], #two-qubit-fidelity,
                                after_reset_flip_probability=0 if not _RESETS else avgs["readout"], #reset error
                                before_measure_flip_probability=avgs["readout"], #measurement error,
                                before_round_data_depolarization=avgs["idle"]) #idle error)
        model = circuit.detector_error_model(decompose_errors=False)

        memory = job.result().get_memory(idx)
        result_kde = cpp_soft_info.decode_IQ_kde(model, memory, 3, int(LOGICAL), _RESETS, 
                                                    qubit_mapping, kde_dict, _DETAILED, relError=rel_error, absError=-1,
                                                    nb_intervals=-1)
        
        num_errors.append(result_kde.num_errors)


cProfile.run("f()")


Decoding circuits: 16it [00:22,  1.41s/it]

         7815 function calls in 22.520 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.010    0.010   22.519   22.519 2418759594.py:3(f)
        1    0.000    0.000    0.000    0.000 <frozen os>:674(__getitem__)
        1    0.000    0.000    0.000    0.000 <frozen os>:756(encode)
        1    0.001    0.001   22.520   22.520 <string>:1(<module>)
      432    0.000    0.000    0.001    0.000 __init__.py:173(search)
      432    0.000    0.000    0.000    0.000 __init__.py:272(_compile)
       64    0.000    0.000    0.001    0.000 _methods.py:101(_mean)
       64    0.000    0.000    0.000    0.000 _methods.py:67(_count_reduce_items)
        1    0.000    0.000    0.000    0.000 _monitor.py:94(report)
        1    0.000    0.000    0.000    0.000 _weakrefset.py:110(remove)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:17(__init__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:21(